In [265]:
import random
import operator
import pandas as pd
from collections import Counter
import nltk
from joblib import Parallel, delayed
import tqdm
import string
import numpy as np
from joblib import Parallel, delayed
import re
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction import FeatureHasher
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
import scipy.sparse
import holidays
import random

In [266]:
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import manhattan_distances
from sklearn.metrics.pairwise import cosine_similarity
import warnings; warnings.simplefilter('ignore')

In [267]:
path_to_data = "/Users/zacharie/Documents/AICCours/MVA/textGraphData/"

In [268]:
training = pd.read_csv(path_to_data + 'training_set.csv', sep=',', header=0).values
training_info = pd.read_csv(path_to_data + 'training_info.csv', sep=',', header=0, parse_dates = [1]).values
test = pd.read_csv(path_to_data + 'test_set.csv', sep=',', header=0).values
test_info = pd.read_csv(path_to_data + 'test_info.csv', sep=',', header=0, parse_dates = [1]).values

In [269]:
integrate_training = []
for each_line in training:
    sender = each_line[0]
    ids = each_line[-1].split(' ')
    for my_id in ids:
        tmp = training_info[training_info[:,0]==int(my_id)]
        if tmp[0,1] > "1998-01-01":
            integrate_training.append([sender,tmp[0,0],tmp[0,1],tmp[0,2], tmp[0,3]])

In [270]:
print len(integrate_training), len(training_info)

43411 43613


In [271]:
integrate_training = pd.DataFrame(integrate_training, columns=['sender','mid','date','body','recipients'])
print integrate_training.index

RangeIndex(start=0, stop=43411, step=1)


In [272]:
integrate_test = []
for each_line in test:
    sender = each_line[0]
    ids = each_line[-1].split(' ')
    for my_id in ids:
        tmp = test_info[test_info[:,0]==int(my_id)]
        integrate_test.append([sender,tmp[0,0],tmp[0,1],tmp[0,2]])

In [273]:
integrate_test = pd.DataFrame(integrate_test, columns=['sender','mid','date','body'])
print integrate_test.index

RangeIndex(start=0, stop=2362, step=1)


In [274]:
print integrate_training['date'].min()
print integrate_training['date'].max()
print integrate_training.index

1998-12-21 05:29:00
2001-11-01 19:12:34
RangeIndex(start=0, stop=43411, step=1)


In [275]:
print integrate_test['date'].min()
print integrate_test['date'].max()
print integrate_test.index

2001-11-02 05:25:29
2002-06-24 13:15:28
RangeIndex(start=0, stop=2362, step=1)


In [276]:
# Counter(integrate_training.sender.tolist())
# local_recs_list = integrate_training['recipients'].tolist()
# recs_temp = [r for rec in local_recs_list for r in rec.split(' ') if '@' in r]
# rec_occ = dict(Counter(recs_temp))
# sorted_rec_occ = sorted(rec_occ.items(), key=operator.itemgetter(1), reverse = True)
# print len(sorted_rec_occ)
# hehe = dict(Counter(integrate_training['sender'].tolist()))
# sorted_sender_occ = sorted(hehe.items(), key=operator.itemgetter(1), reverse = True)
# print sorted_sender_occ
# hehe = dict(Counter(integrate_test['sender'].tolist()))
# sorted_sender_occ = sorted(hehe.items(), key=operator.itemgetter(1), reverse = True)
# print sorted_sender_occ
print set(integrate_test['sender'].tolist())== set(integrate_training['sender'].tolist())

True


In [277]:
print integrate_training['body'].index
print integrate_test['body'].index

RangeIndex(start=0, stop=43411, step=1)
RangeIndex(start=0, stop=2362, step=1)


In [278]:
integrate_training['date'] = integrate_training['date'].apply(pd.to_datetime)

In [279]:
#integrate_test
print integrate_training['date'].min()
print integrate_training['date'].max()
print integrate_test['date'].min()
print integrate_test['date'].max()

1998-12-21 05:29:00
2001-11-01 19:12:34
2001-11-02 05:25:29
2002-06-24 13:15:28


In [280]:
# remove dashes and apostrophes from punctuation marks
punct = string.punctuation.replace('-', '').replace("'",'')
stpwds = nltk.corpus.stopwords.words('english')
# regex to match intra-word dashes and intra-word apostrophes
my_regex = re.compile(r"(\b[-']\b)|[\W_]")
stemmering = nltk.PorterStemmer()
# performs basic pre-processing
# note: we do not lowercase for consistency with Google News embeddings
def clean_string(string, punct=punct, my_regex=my_regex):
    # remove formatting
    str = re.sub('\s+', ' ', string)
    # remove punctuation (preserving dashes)
    str = ''.join(l for l in str if l not in punct)
    # remove dashes that are not intra-word
    str = my_regex.sub(lambda x: (x.group(1) if x.group(1) else ' '), str)
    # strip extra white space
    str = re.sub(' +',' ',str)
    # strip leading and trailing white space
    str = str.strip()
    # tokenize (split based on whitespace)
    tokens = str.split(' ')
    # remove stopwords
    tokens = [token for token in tokens if token not in stpwds]
    # remove tokens less than 2 characters in size
    # tokens = [token for token in tokens if len(token)>=2]
    tokens = [stemmering.stem(token.lower()) for token in tokens if len(token)>=2]
    return ' '.join(tokens)

def stemmering_sentences_messages(X):
    sentences_stem = Parallel(n_jobs=4)(delayed(clean_string)(sentence) for sentence in tqdm.tqdm(X, desc="clean sentences"))
    return sentences_stem


In [281]:
integrate_training['body'] = stemmering_sentences_messages(integrate_training['body'])

clean sentences: 100%|██████████| 43411/43411 [01:30<00:00, 477.98it/s]


In [282]:
integrate_test['body'] = stemmering_sentences_messages(integrate_test['body'])

clean sentences: 100%|██████████| 2362/2362 [00:05<00:00, 434.91it/s]


In [283]:
# us_holidays = holidays.UnitedStates()
# Counter(integrate_test['date'].apply(lambda x: int(x in us_holidays)))

In [284]:
# integrate_test['holiday'] = integrate_test['date'].apply(lambda x: int(x in us_holidays))
# integrate_training['holiday'] = integrate_training['date'].apply(lambda x: int(x in us_holidays))

In [285]:
# integrate_test['quarter'] = integrate_test['date'].dt.quarter
# integrate_training['quarter'] = integrate_training['date'].dt.quarter

In [286]:
integrate_training['month'] = integrate_training['date'].dt.month.map(str)
integrate_training['day'] = integrate_training['date'].dt.day.map(str)
integrate_training['hour'] = integrate_training['date'].dt.hour.map(str)
integrate_training['minute'] = integrate_training['date'].dt.minute.map(str)
integrate_training['date_body'] = integrate_training[['month', 'day', 'hour', 'minute', 'body']].apply(lambda x: ' '.join(x), axis=1)

In [287]:
integrate_test['month'] = integrate_test['date'].dt.month.map(str)
integrate_test['day'] = integrate_test['date'].dt.day.map(str)
integrate_test['hour'] = integrate_test['date'].dt.hour.map(str)
integrate_test['minute'] = integrate_test['date'].dt.minute.map(str)
integrate_test['date_body'] = integrate_test[['month', 'day', 'hour', 'minute', 'body']].apply(lambda x: ' '.join(x), axis=1)

In [288]:
# integrate_test.drop(labels=['date', 'body', 'month', 'day', 'hour', 'minute'], axis=1, inplace=True)


In [289]:
print integrate_test.columns
print integrate_training.columns


Index([u'sender', u'mid', u'date', u'body', u'month', u'day', u'hour',
       u'minute', u'date_body'],
      dtype='object')
Index([u'sender', u'mid', u'date', u'body', u'recipients', u'month', u'day',
       u'hour', u'minute', u'date_body'],
      dtype='object')


In [290]:
# integrate_training_1 = pd.get_dummies(integrate_training,columns=['sender'])
# integrate_test_1 = pd.get_dummies(integrate_test,columns=['sender'])
# integrate_training_1.insert(0, 'sender', integrate_training['sender'])
# integrate_test_1.insert(0, 'sender', integrate_test['sender'])
# integrate_training = integrate_training_1
# integrate_test = integrate_test_1

In [291]:
# integrate_training = pd.get_dummies(integrate_training, prefix=['q','m','d','h'], columns=['quarter', 'month', 'day', 'hour'])
# integrate_test = pd.get_dummies(integrate_test, prefix=['q','m','d','h'], columns=['quarter', 'month', 'day', 'hour'])

In [292]:
print integrate_training.shape
print integrate_test.shape

(43411, 10)
(2362, 9)


In [293]:
X = integrate_training['date_body'].tolist() + integrate_test['date_body'].tolist()

In [294]:
vect = TfidfVectorizer(min_df=10, max_df=0.9,stop_words='english',analyzer='word', norm='l2', lowercase=True, tokenizer=nltk.word_tokenize, ngram_range=(1, 1))
X_sentences = vect.fit_transform(X)


In [295]:
print X_sentences.shape

(45773, 23009)


In [296]:
integrate_training_1 = pd.get_dummies(integrate_training,columns=['sender'])
integrate_test_1 = pd.get_dummies(integrate_test,columns=['sender'])
integrate_training_1.insert(0, 'sender', integrate_training['sender'])
integrate_test_1.insert(0, 'sender', integrate_test['sender'])
integrate_training = integrate_training_1
integrate_test = integrate_test_1

In [297]:
print integrate_training.columns
print integrate_test.columns

Index([u'sender', u'mid', u'date', u'body', u'recipients', u'month', u'day',
       u'hour', u'minute', u'date_body',
       ...
       u'sender_susan.scott@enron.com', u'sender_suzanne.adams@enron.com',
       u'sender_sylvia.hu@enron.com', u'sender_tanya.rohauer@enron.com',
       u'sender_taylor@enron.com', u'sender_tim.belden@enron.com',
       u'sender_tori.kuykendall@enron.com', u'sender_vkaminski@aol.com',
       u'sender_w..cantrell@enron.com', u'sender_wsmith@wordsmith.org'],
      dtype='object', length=135)
Index([u'sender', u'mid', u'date', u'body', u'month', u'day', u'hour',
       u'minute', u'date_body', u'sender_alan.aronowitz@enron.com',
       ...
       u'sender_susan.scott@enron.com', u'sender_suzanne.adams@enron.com',
       u'sender_sylvia.hu@enron.com', u'sender_tanya.rohauer@enron.com',
       u'sender_taylor@enron.com', u'sender_tim.belden@enron.com',
       u'sender_tori.kuykendall@enron.com', u'sender_vkaminski@aol.com',
       u'sender_w..cantrell@enron.com'

In [298]:
tmp_sender_array = scipy.sparse.vstack([integrate_training.iloc[:, 10:].as_matrix(), integrate_test.iloc[:, 9:].as_matrix()])
print tmp_sender_array.shape

(45773, 125)


In [299]:
X_sentences_4 = scipy.sparse.hstack([X_sentences, tmp_sender_array])

In [300]:
import pickle
with open('./' + 'processed_X_sentences_4.pkl', 'wb') as f:
    pickle.dump(X_sentences_4, f)

In [301]:
import pickle
with open('./' + 'processed_X_sentences_4.pkl', 'rb') as f:
    X_sentences_4 = pickle.load(f)

In [302]:
# svd = TruncatedSVD(n_components=500, random_state=42)
# X_sentences_reduced = svd.fit_transform(X_sentences_1)

In [303]:
len_training = integrate_training.shape[0]
print len_training
len_test = integrate_test.shape[0]
print len_test

43411
2362


In [304]:
X_training = X_sentences_4.toarray()[:len_training,:]
X_test = X_sentences_4.toarray()[len_training:, :]

In [305]:
print len(X_training) == len_training
print len(X_test) == len_test

True
True


In [ ]:
neigh = NearestNeighbors(n_neighbors=30, radius=2.0, n_jobs = 4)
# neigh = NearestNeighbors(n_neighbors=30, metric=cosine_similarity, n_jobs=4)
neigh.fit(X_training)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=4, n_neighbors=30, p=2, radius=2.0)

In [ ]:
distance, index = neigh.kneighbors(X_test)

In [ ]:
integrate_training['sender'].index

In [ ]:
predictions_per_test_mid = []
k_constant=10
for i , series in integrate_test.iterrows():
    row = series.tolist()
    sender = row[0]
    mid = row[1]
    index_per_sender = integrate_training[integrate_training['sender'] == sender].index.tolist()
    index_total = np.intersect1d(np.array(index_per_sender), index[i])
    local_recs_list = integrate_training.iloc[index_total]['recipients'].tolist()
    recs_temp = [r for rec in local_recs_list for r in rec.split(' ') if '@' in r]
    rec_occ = dict(Counter(recs_temp))
    sorted_rec_occ = sorted(rec_occ.items(), key=operator.itemgetter(1), reverse = True)
    k_most = [tuple[0] for tuple in sorted_rec_occ[:k_constant]]
    if len(k_most) <= k_constant:
        print '!'
    predictions_per_test_mid.append([mid, k_most])

In [ ]:
path_to_results = "/Users/zacharie/Documents/AICCours/MVA/textGraphData/"
with open(path_to_results + 'predictions_frequency_11.txt', 'wb') as my_file:
    my_file.write('mid,recipients' + '\n')
    for mid_k_most in predictions_per_test_mid:
        my_file.write(str(mid_k_most[0]) + ',' + ' '.join(mid_k_most[-1]) + '\n')

In [ ]:
# predictions_per_test_mid = []
# k_constant=10
# for line in test:
#     sender = line[0]
#     print sender
#     index_per_sender_training = np.array(integrate_training[integrate_training['sender'] == sender].index.tolist())
#     nei = NearestNeighbors(n_neighbors=20, radius=2.0, n_jobs = -1)
#     nei.fit(X_training[index_per_sender_training,])
#     index_per_sender_test = integrate_test[integrate_test['sender'] == sender].index.tolist()
#     mids = integrate_test[integrate_test['sender'] == sender]['mid'].tolist()
#     neigh_test_index = nei.kneighbors(X_test[index_per_sender_test, ], return_distance=False)
#     for i, neigh_index in enumerate(neigh_test_index):
#         local_recs_list = integrate_training.iloc[index_per_sender_training[neigh_index]]['recipients'].tolist()
#         recs_temp = [r for rec in local_recs_list for r in rec.split(' ') if '@' in r]
#         rec_occ = dict(Counter(recs_temp))
#         # order by frequency
#         sorted_rec_occ = sorted(rec_occ.items(), key=operator.itemgetter(1), reverse = True)
#         k_most = [tuple[0] for tuple in sorted_rec_occ[:k_constant]]
#         predictions_per_test_mid.append([mids[i], k_most])